Start jupyter server such that the print of the current working dir below is the root directory of the repo
Setup the environment...

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import sys
os.environ['STATIC_VARIABLES_FILE_PATH'] = "globalserver/static_variables.json"
os.environ['PATH_TO_GLOBALSERVER'] = "globalserver/api/"
if os.getcwd().split('/')[-1]!="Federated-Learning":
    os.chdir('..')
print(os.getcwd())
sys.path.append(os.getcwd())

import tensorflow_addons as tfa

from testing.test_class import Testing
from globalserver.operator_.operator_class_db import Operator

clients = [f"c{i}" for i in range(2)]
TestSetup = Testing(clients, start_servers=True, clear_logs=True, clear_db=False, interface=False)


/home/schneech/federated_learning/Federated-Learning


2020-03-12 13:43:44,602 [MainProcess ] [INFO ] [test] [test_class.py / kill_servers / 160] KILLING SERVERS
2020-03-12 13:43:44,604 [MainProcess ] [INFO ] [test] [test_class.py / kill_global_server / 122] KILLING GLOBAL SERVER
2020-03-12 13:43:44,605 [MainProcess ] [INFO ] [test] [test_class.py / kill_client_interface_node / 129] KILLING CLIENT INTERFACE NODE
2020-03-12 13:43:44,607 [MainProcess ] [INFO ] [test] [test_class.py / kill_client_interface / 139] KILLING CLIENT INTERFACE
2020-03-12 13:43:44,608 [MainProcess ] [INFO ] [test] [test_class.py / kill_node_servers / 149] KILLINGCLIENTS
2020-03-12 13:43:47,724 [MainProcess ] [INFO ] [test] [test_class.py / start_global_server / 303] STARTING UP GLOBAL SERVER
2020-03-12 13:43:47,753 [MainProcess ] [INFO ] [test] [test_class.py / start_node_servers / 191] STARTING UP NODES


the commandline is ['/home/schneech/federated_learning/federated_learning/bin/python', 'api/globalserver_task_controller.py', 'C88A33B946']
the commandline is ['/home/schneech/federated_learning/federated_learning/bin/python', 'node_task_controller.py', 'c0', 'C88A33B946']
the commandline is ['/home/schneech/federated_learning/federated_learning/bin/python', 'node_task_controller.py', 'c1', 'C88A33B946']


Download the data from polybox https://polybox.ethz.ch/index.php/s/W1oSh3H81HqYSQp?path=%2FDatasets%2FKK_Box_Normalized_jsonl and put it into the datasets/kkbox folder. Then split the data into two sets of size 70000.

In [2]:
import random
import json

def split_data():
    
    with open(f'datasets/kk_box/train_kkbox.jsonl', 'r') as f:
        train=f.readlines()
    with open(f'datasets/kk_box/test_kkbox.jsonl', 'r') as f:
        test=f.readlines()
    random.shuffle(train)
    train_c1 = train[:70000]
    train_c2 = train[70000:140000]
    validation_c1 = train[140000:160000]
    validation_c2 = train[160000:180000]

    with open(f'datasets/train_c0.jsonl', 'w+') as f:
        f.writelines(train_c1)
    with open(f'datasets/validation_c0.jsonl', 'w+') as f:
        f.writelines(validation_c1)
    with open(f'datasets/test_c0.jsonl', 'w+') as f:
        f.writelines(validation_c1)

    with open(f'datasets/train_c1.jsonl', 'w+') as f:
        f.writelines(train_c2)
    with open(f'datasets/validation_c1.jsonl', 'w+') as f:
        f.writelines(validation_c2)
    with open(f'datasets/test_c1.jsonl', 'w+') as f:
        f.writelines(validation_c2)
        
# split_data()




We define the experimental setup in several steps. First we define the model.

In [3]:
def kknox_nn(param_dict):
    model = Sequential()
    model.add(Dense(50, activation=tf.nn.relu, input_dim=61))
    model.add(Dense(50, activation=tf.nn.relu))
    model.add(Dense(1, activation=tf.nn.sigmoid))

    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=param_dict['lr']),  # momentum=mt),
                  loss='binary_crossentropy',
                  metrics=[])
    model.summary()
    return model

We define all the metadata as well as the training configurations. Most important: rounds, epochs, batch_size, steps_per_epoch.

In [10]:
setup_dict = {"model_function": #here we set how to run our model function. In the operator it will run kknox_nn({"lr": 0.1})
              {
    "function": kknox_nn,
    "parameters": {
        "lr": 0.20014587438532283
    }
},
    "git_version": 'e9339081b76ad3a89b1862bd38d8af26f0541f1c', # the git commit version (for now set manually)
    "protocol": 'NN', # the protocol name. NN is for neural networks
    "model_name": "test_model",
    "model_description": "this model is just to test the db",
    "testing": True, # This flag is helpful if you do not want to keep the created models. When you run Testing with cleardb=True it will clear all models and experiments with testing flag.
    "rounds": 2,
    "round": ["fetch_model", "train_model", "send_training_loss", "send_validation_loss", "send_model", "aggregate"],
    "final_round":["fetch_model", "send_training_loss", "send_validation_loss","send_test_loss"],
    "training_config": { # standard configuration for the training setup
        'epochs': 1,
        'verbose': 1,
        'batch_size': 2000,
        "validation_steps": 8,#should be at most len(validation data)/batch_size
        "test_steps": 8, #should be at most  len(test data)/batch_size
        "steps_per_epoch": 65, #should be at most  len(training data)/batch_size
        "dataset": "",

        "skmetrics":["f1_score"],
        "tfmetrics":["AUC"]
    },
   "clients": clients,
    "experiment_name": "kkbox",
    "experiment_description": f"desc if nice experiment",
        "preprocessing": {
    },
}

In [11]:

operator = Operator()
operator.define_and_start_experiment(setup_dict)


2020-03-12 13:45:30,604 [MainProcess ] [DEBUG] [test] [cmd.py / execute / 719] Popen(['git', 'cat-file', '--batch-check'], cwd=/home/schneech/federated_learning/Federated-Learning, universal_newlines=False, shell=None, istream=<valid stream>)
2020-03-12 13:45:30,624 [MainProcess ] [INFO ] [test] [operator_class_db.py / start_experiment / 170] 5e6a2eeaf8219093ef163769
2020-03-12 13:45:30,626 [MainProcess ] [DEBUG] [test] [operator_class_db.py / start_experiment / 182] Working on {'task_id': ObjectId('5e6a2eeaf8219093ef16376a'), 'task_order': 0, 'task_name': 'fetch_model', 'task_status': 'not_scheduled'}


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 50)                3100      
_________________________________________________________________
dense_7 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 51        
Total params: 5,701
Trainable params: 5,701
Non-trainable params: 0
_________________________________________________________________


2020-03-12 13:45:45,641 [MainProcess ] [DEBUG] [test] [operator_class_db.py / start_experiment / 182] Working on {'task_id': ObjectId('5e6a2eeaf8219093ef16376b'), 'task_order': 1, 'task_name': 'train_model', 'task_status': 'scheduled'}
2020-03-12 13:45:50,646 [MainProcess ] [DEBUG] [test] [operator_class_db.py / start_experiment / 182] Working on {'task_id': ObjectId('5e6a2eeaf8219093ef16376d'), 'task_order': 3, 'task_name': 'send_validation_loss', 'task_status': 'scheduled'}
2020-03-12 13:45:55,654 [MainProcess ] [DEBUG] [test] [operator_class_db.py / start_experiment / 182] Working on {'task_id': ObjectId('5e6a2eeaf8219093ef16376f'), 'task_order': 5, 'task_name': 'aggregate', 'task_status': 'not_scheduled'}
2020-03-12 13:45:55,657 [MainProcess ] [INFO ] [test] [operator_class_db.py / aggregate / 230] Aggregating...
2020-03-12 13:45:55,686 [MainProcess ] [INFO ] [test] [operator_utils.py / NN_aggregate_model_updates / 245] Warning: Weight matrix is getting big: 7.446819592788565
2020-

(ObjectId('5e6a2eeaf8219093ef163769'),
 [{'aggregated_metric': {'f1_score': 0.0, 'AUC': 0.6789364218711853},
   'additional_metrics': []},
  {'aggregated_metric': {'f1_score': 0.0, 'AUC': 0.7090849578380585},
   'additional_metrics': []}])